# Multi-table Datasets - ENRON Archive

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

> Answer:
There are 3 tables:
- MessageBase
- RecipientBase
- EmployeeBase

In [7]:
import sqlite3
conn = sqlite3.connect('../../assets/datasets/enron.db') 
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[(u'MessageBase',), (u'RecipientBase',), (u'EmployeeBase',)]

Query the `sqlite_master` table to retrieve the schema of the `EmployeeBase` table.

1. What fields are there?
1. What's the type of each of them?

In [28]:
fields = c.execute("SELECT sql FROM sqlite_master WHERE type='table' and name='RecipientBase';").fetchall()
print ''.join(fields[0])
fields = c.execute("SELECT sql FROM sqlite_master WHERE type='table' and name='MessageBase';").fetchall()
print ''.join(fields[0])
fields = c.execute("SELECT sql FROM sqlite_master WHERE type='table' and name='EmployeeBase';").fetchall()
print ''.join(fields[0])

CREATE TABLE RecipientBase (
    mid INTEGER,
    rno INTEGER,
    to_eid INTEGER,
    
    PRIMARY KEY(mid ASC, rno ASC)
    FOREIGN KEY(mid) REFERENCES Message(mid)
    FOREIGN KEY(to_eid) REFERENCES Employee(eid)
)
CREATE TABLE MessageBase (
    mid INTEGER,
    filename TEXT,
    unix_time INTEGER,
    subject TEXT,
    from_eid INTEGER,
    
    PRIMARY KEY(mid ASC),
    FOREIGN KEY(from_eid) REFERENCES Employee(eid)
)
CREATE TABLE EmployeeBase (
                  [eid] INTEGER,
  [name] TEXT,
  [department] TEXT,
  [longdepartment] TEXT,
  [title] TEXT,
  [gender] TEXT,
  [seniority] TEXT
                  
                  )


1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

In [33]:
results = c.execute("SELECT * FROM RecipientBase LIMIT 5;").fetchall()
for row in results:
    print row

(1, 1, 59)
(2, 1, 15)
(3, 1, 15)
(4, 1, 109)
(4, 2, 49)


In [34]:
results = c.execute("SELECT * FROM EmployeeBase LIMIT 5;").fetchall()
for row in results:
    print row

(1, u'John Arnold', u'Forestry', u'ENA Gas Financial', u'VP Trading', u'Male', u'Senior')
(2, u'Harry Arora', u'Forestry', u'ENA East Power', u'VP Trading', u'Male', u'Senior')
(3, u'Robert Badeer', u'Forestry', u'ENA West Power', u'Mgr Trading', u'Male', u'Junior')
(4, u'Susan Bailey', u'Legal', u'ENA Legal', u'Specialist Legal', u'Female', u'Junior')
(5, u'Eric Bass', u'Forestry', u'ENA Gas Texas', u'Trader', u'Male', u'Junior')


In [35]:
results = c.execute("SELECT * FROM MessageBase LIMIT 5;").fetchall()
for row in results:
    print row

(1, u'taylor-m/sent/11', 910930020, u'Cd$ CME letter', 138)
(2, u'taylor-m/sent/17', 911459940, u'Indemnification', 138)
(3, u'taylor-m/sent/18', 911463840, u'Re: Indemnification', 138)
(4, u'taylor-m/sent/23', 911874180, u'Re: Coral Energy, L.P.', 138)
(5, u'taylor-m/sent/27', 912396120, u'Bankruptcy Code revisions', 138)


Import each of the 3 tables to a Pandas Dataframes

In [45]:
import pandas as pd
employees = pd.read_sql("SELECT * FROM EmployeeBase;", conn)
recipients = pd.read_sql("SELECT * FROM RecipientBase;", conn)
messages = pd.read_sql("SELECT * FROM MessageBase;", conn)

## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company?
- How many messages are there in the database?
- Convert the timestamp column in the messages. When was the oldest message sent? And the newest?
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - How many messages have only one recepient?
    - How many messages have >= 5 recepients?
    - What's the highest number of recepients?
    - Who sent the message with the highest number of recepients?
- Plot the distribution of recepient numbers using Bokeh.

In [106]:
sql.read_sql('SELECT Count(eid) FROM EmployeeBase', con=conn)

,Count(eid)
0,156


In [107]:
sql.read_sql('SELECT Count(mid) FROM MessageBase', con=conn)

,Count(mid)
0,21635


In [108]:
sql.read_sql('SELECT Count(eid) FROM EmployeeBase', con=conn)

,Count(eid)
0,156


In [112]:
datetimes = messages['unix_time'].apply(pd.datetime.fromtimestamp)
print "first msg was sent on:", min(datetimes)
print "last msg was sent on:", max(datetimes)

first msg was sent on: 1998-11-12 23:07:00
last msg was sent on: 2002-06-21 09:37:34


In [113]:
from collections import Counter

counts = Counter(recipients.groupby('mid')['to_eid'].count())
counts

Counter({1: 14985,
         2: 2962,
         3: 1435,
         4: 873,
         5: 711,
         6: 180,
         7: 176,
         8: 61,
         9: 24,
         10: 29,
         11: 47,
         12: 33,
         13: 57,
         14: 11,
         15: 28,
         16: 9,
         17: 2,
         18: 1,
         19: 1,
         20: 1,
         21: 2,
         22: 1,
         24: 1,
         49: 1,
         52: 1,
         55: 1,
         57: 2})

In [114]:
from bokeh.plotting import figure,show,output_notebook
output_notebook()

Loading BokehJS ...

In [115]:
x = [i[0] for i in counts.most_common()]
y = [i[1] for i in counts.most_common()]
left_border = [val-0.5 for val in x]
right_border = [val+0.5 for val in x]

In [116]:
p= figure(title="Message Recipients",tools='',x_axis_label='# of recipients',y_axis_label='Counts')
p.quad(top=y,left=left_border,right=right_border,bottom=0,line_color='black')
show(p)

Rescale to investigate the tail of the curve

In [117]:
x = [i[0] for i in counts.most_common()[5:]]  # chop off the first 5
y = [i[1] for i in counts.most_common()[5:]]  # chop off the first 5
left_border = [val-0.5 for val in x]
right_border = [val+0.5 for val in x]


p= figure(title="Message Recipients",tools='',x_axis_label='# of recipients',y_axis_label='Counts')
p.quad(top=y,left=left_border,right=right_border,bottom=0,line_color='black')
show(p)

## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)

In [118]:
employees.gender.value_counts()

Male      113
Female     43
Name: gender, dtype: int64

In [119]:
employees.groupby('department')['gender'].value_counts()

department  gender
Forestry    Male      50
            Female    10
Legal       Female    13
            Male      12
Other       Male      51
            Female    20
Name: gender, dtype: int64

In [120]:
df = pd.merge(employees, messages, left_on='eid', right_on='from_eid')
df.gender.value_counts() / df.gender.count()

Male      0.593529
Female    0.406471
Name: gender, dtype: float64

In [121]:
# What's the average number of emails sent by each gender?
df.gender.value_counts() / employees.gender.value_counts()


Male      113.637168
Female    204.511628
Name: gender, dtype: float64

In [122]:
employees.seniority.value_counts()

Junior    82
Senior    74
Name: seniority, dtype: int64

In [123]:
df.seniority.value_counts()


Senior    12439
Junior     9196
Name: seniority, dtype: int64

In [124]:
df.seniority.value_counts() / employees.seniority.value_counts()


Junior    112.146341
Senior    168.094595
Name: seniority, dtype: float64

In [125]:
# Which department is sending more emails? How does that relate with the number of employees in the department?
df.department.value_counts()

Legal       10396
Other        6852
Forestry     4387
Name: department, dtype: int64

In [126]:
df.department.value_counts() / employees.department.value_counts()


Forestry     73.116667
Legal       415.840000
Other        96.507042
Name: department, dtype: float64

In [127]:
# Who are the top 5 senders of emails? (people who sent out the most emails)
top5senders = df.eid.value_counts().head().reset_index()
top5senders.columns = ['eid', 'msgs_sent']
top5senders

,eid,msgs_sent
0,20,1597
1,59,1379
2,120,1142
3,131,859
4,138,658


In [128]:
pd.merge(employees, top5senders, on='eid')


,eid,name,department,longdepartment,title,gender,seniority,msgs_sent
0,20,Jeff Dasovich,Legal,Regulatory and Government Affairs,Director,Male,Senior,1597
1,59,Tana Jones,Legal,ENA Legal,Specialist Legal,Female,Junior,1379
2,120,Sara Shackleton,Legal,ENA Legal,Gen Cnsl Asst,Female,Junior,1142
3,131,James D. Steffes,Legal,Regulatory and Government Affairs,VP of Government Affairs,Male,Senior,859
4,138,Mark E. Taylor,Legal,ENA Legal,VP & Gen Cnsl,Male,Senior,658


Answer the following questions regarding received messages:

- Who is receiving more emails? Men or Women?
- Who is receiving more emails? Juniors or Seniors?
- Which department is receiving more emails? How does that relate with the number of employees in the department?
- Who are the top 5 receivers of emails? (people who received the most emails)

Which employees sent the most 'mass' emails?

Keep exploring the dataset, which other questions would you ask?

Work in pairs. Give each other a challenge and try to solve it.